In [34]:
#PYTHON 3.10.11

import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.interpolate import make_interp_spline


from imblearn.combine import SMOTEENN # oversampling es undersamplingre

from imblearn.over_sampling import SMOTE

#models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.base import BaseEstimator
from imblearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC


#Hyperparameter-tuning
from sklearn.model_selection import train_test_split, GridSearchCV
from skopt.searchcv import BayesSearchCV

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
from sklearn.tree import plot_tree

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, auc

from sklearn.metrics import f1_score, recall_score, accuracy_score

In [33]:
%pip install lightgbm 
%pip install xgboost 

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 1.4/1.4 MB 15.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   -- ------------------------------------- 6.3/124.9 MB 38.6 MB/s eta 0:00:04
   ----- ---------------------------------- 15.7/124.9 MB 41.2 MB/s eta 0:00:03
   ------- -------------------------------- 22.0/124.9 MB 42.2 MB/s eta 0:00:03
   ------- -------------------------------- 23.1/124.9 MB 28.6 MB/s eta 0:00:04
   ---------- ----------------------------- 32.5/124.9 MB 32.2 MB/s eta 0:00:03
   ------------- -------------------------- 41.4/124.9 MB 33.8 MB/s eta 0:00:03
   ---------------- ----------------------- 50.6/124.9 MB 35.4 MB/s eta 0:00:03
   ------------------- -------------------- 60.0/124.9 MB 36.4 MB/s eta 0:00:02
   ---------------------- ----------------- 69.7/124.9 MB 37.4 MB/s eta 0:00:0

In [ ]:
df = pd.read_csv("heart.csv")
df.columns = df.columns.str.strip()
df.columns

X = df.drop("output", axis=1)

X = df.drop(columns=["output", "chol", "fbs", "restecg"])

y = df["output"]

In [ ]:
models_params_gridsearch = {
    "XGBoost": {
        "model": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
        "params": {
            "n_estimators": [50, 100, 200],
            "max_depth": [3, 5, 7],
            "learning_rate": [0.01, 0.1, 0.2],
            "subsample": [0.7, 0.8, 1.0]
        }
    },
    "LightGBM": {
        "model": LGBMClassifier(random_state=42),
        "params": {
            "n_estimators": [50, 100, 200],
            "num_leaves": [20, 31, 40],
            "max_depth": [-1, 5, 10],
            "learning_rate": [0.01, 0.1, 0.2]
        }
    },
    "KNeighborsClassifier": {
        "model": KNeighborsClassifier(),
        "params": {
            "n_neighbors": [3, 5, 7, 9, 11]
        }
    },
    "RandomForestClassifier": {
        "model": RandomForestClassifier(),
        "params": {
            "n_estimators": [10, 50, 100, 200],
            "max_depth": [None, 5, 10, 15, 20]
        }
    },
    "DecisionTreeClassifier": {
        "model": DecisionTreeClassifier(),
        "params": {
            "max_depth": [None, 5, 10, 15, 20]
        }
    },
    "SVC": {
        "model": SVC(probability=True),
        "params": {
            "C": [0.1, 1, 10, 100],
            "gamma": ["scale", "auto"],
            "kernel": ["linear", "rbf"]
        }
    },
    "LogisticRegression": {
        "model": LogisticRegression(),
        "params": {
            "C": [0.1, 1, 10, 100],
            'solver': ['liblinear', 'lbfgs', 'newton-cg'],
            'penalty': ['l2', 'elasticnet'],
            'l1_ratio': [0.5, 0.7, 0.9]
        }
    }
}

In [23]:
from sklearn.model_selection import StratifiedKFold, train_test_split, GridSearchCV
from sklearn.metrics import f1_score, recall_score, accuracy_score, roc_auc_score, classification_report, confusion_matrix, roc_curve
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import numpy as np
import json

# Define a pipeline function
def evaluate_models_pipeline(X, y, models_params_gridsearch, smote_strategy=0.85, test_size=0.2):
    results = []

    # Step 1: Resample Data
    smote = SMOTE(random_state=32, sampling_strategy=smote_strategy)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    # Step 2: Train-test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X_resampled, y_resampled, test_size=test_size, random_state=32, stratify=y_resampled
    )

    # Step 3: Scale Data
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Step 4: Define StratifiedKFold
    stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    best_model = None
    best_params = None
    best_score = 0
    best_accuracy = 0
    best_conf_mx = None
    best_class_report = None
    best_roc_auc = 0

    for model_name, mp in models_params_gridsearch.items():
        clf = GridSearchCV(
            mp['model'],
            mp['params'],
            cv=stratified_kfold,
            scoring='roc_auc',
            return_train_score=False,
            n_jobs=-1
        )

        # Step 5: Fit the Model
        clf.fit(X_train_scaled, y_train)
        model = clf.best_estimator_
        model.fit(X_train_scaled, y_train)

        # Step 6: Evaluate the Model
        y_pred = model.predict(X_test_scaled)
        y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

        conf_mx = confusion_matrix(y_test, y_pred, labels=[0, 1])
        class_report = classification_report(y_test, y_pred, target_names=['lower chance of heart disease', 'higher chance of heart disease'], digits=5)

        roc_auc = roc_auc_score(y_test, y_pred_proba)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)

        avg_score = (roc_auc + recall) / 2

        # Update best model
        if avg_score > best_score:
            best_score = avg_score
            best_model = model_name
            best_params = clf.best_params_
            best_accuracy = accuracy
            best_conf_mx = conf_mx.tolist()
            best_class_report = class_report
            best_roc_auc = roc_auc

    # Save best result
    best_result = {
        "smote": smote_strategy,
        "test_size": test_size,
        "model_name": best_model,
        "best_params": best_params,
        "accuracy": best_accuracy,
        "avg_score": best_score,
        "confusion_matrix": best_conf_mx,
        "roc_auc": best_roc_auc,
        "classification_report": best_class_report
    }

    return best_result


In [30]:

# Loop over parameter ranges
smote_range = np.arange(0.85, 0.9, 0.01)
test_size_range = np.arange(0.1, 0.3, 0.05)

best_accuracy = 0
best_smote = 0
best_test_size = 0

all_results = []
for smote_strategy in smote_range:
    for test_size in test_size_range:
        print(f"smote = {smote_strategy} test_size = {test_size}")
        best_result = evaluate_models_pipeline(X, y, models_params_gridsearch, smote_strategy, test_size)
        best_result["smote_strategy"] = smote_strategy
        best_result["test_size"] = test_size
        if(best_result["accuracy"] > best_accuracy):
            best_smote = best_result["smote_strategy"]
            best_test_size = best_result["test_size"]
        print(best_result)
        all_results.append(best_result)

print(f"best acc= {best_accuracy}\nbest test_size= {best_test_size}\nbest smote= {best_smote}")
# Save all results to a JSON file
output_file = "model_results.json"
with open(output_file, "w") as f:
    json.dump(all_results, f, indent=4)


smote = 0.85 test_size = 0.1


C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packa

{'smote': 0.85, 'test_size': 0.1, 'model_name': 'KNeighborsClassifier', 'best_params': {'n_neighbors': 11}, 'accuracy': 0.8064516129032258, 'avg_score': 0.8728991596638656, 'confusion_matrix': [[10, 4], [2, 15]], 'roc_auc': 0.8634453781512604, 'classification_report': '                                precision    recall  f1-score   support\n\n lower chance of heart disease    0.83333   0.71429   0.76923        14\nhigher chance of heart disease    0.78947   0.88235   0.83333        17\n\n                      accuracy                        0.80645        31\n                     macro avg    0.81140   0.79832   0.80128        31\n                  weighted avg    0.80928   0.80645   0.80438        31\n', 'smote_strategy': 0.85}
smote = 0.85 test_size = 0.15000000000000002


C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packa

{'smote': 0.85, 'test_size': 0.15000000000000002, 'model_name': 'SVC', 'best_params': {'C': 0.1, 'gamma': 'auto', 'kernel': 'rbf'}, 'accuracy': 0.8478260869565217, 'avg_score': 0.9095238095238096, 'confusion_matrix': [[16, 5], [2, 23]], 'roc_auc': 0.8990476190476191, 'classification_report': '                                precision    recall  f1-score   support\n\n lower chance of heart disease    0.88889   0.76190   0.82051        21\nhigher chance of heart disease    0.82143   0.92000   0.86792        25\n\n                      accuracy                        0.84783        46\n                     macro avg    0.85516   0.84095   0.84422        46\n                  weighted avg    0.85223   0.84783   0.84628        46\n', 'smote_strategy': 0.85}
smote = 0.85 test_size = 0.20000000000000004


...

C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packa

{'smote': 0.9, 'test_size': 0.15000000000000002, 'model_name': 'RandomForestClassifier', 'best_params': {'max_depth': 5, 'n_estimators': 200}, 'accuracy': 0.7872340425531915, 'avg_score': 0.8445454545454545, 'confusion_matrix': [[16, 6], [4, 21]], 'roc_auc': 0.8490909090909091, 'classification_report': '                                precision    recall  f1-score   support\n\n lower chance of heart disease    0.80000   0.72727   0.76190        22\nhigher chance of heart disease    0.77778   0.84000   0.80769        25\n\n                      accuracy                        0.78723        47\n                     macro avg    0.78889   0.78364   0.78480        47\n                  weighted avg    0.78818   0.78723   0.78626        47\n', 'smote_strategy': 0.9}
smote = 0.9 test_size = 0.20000000000000004


C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packa

{'smote': 0.9, 'test_size': 0.20000000000000004, 'model_name': 'KNeighborsClassifier', 'best_params': {'n_neighbors': 11}, 'accuracy': 0.8095238095238095, 'avg_score': 0.876010101010101, 'confusion_matrix': [[22, 8], [4, 29]], 'roc_auc': 0.8732323232323232, 'classification_report': '                                precision    recall  f1-score   support\n\n lower chance of heart disease    0.84615   0.73333   0.78571        30\nhigher chance of heart disease    0.78378   0.87879   0.82857        33\n\n                      accuracy                        0.80952        63\n                     macro avg    0.81497   0.80606   0.80714        63\n                  weighted avg    0.81348   0.80952   0.80816        63\n', 'smote_strategy': 0.9}
smote = 0.9 test_size = 0.25000000000000006
{'smote': 0.9, 'test_size': 0.25000000000000006, 'model_name': 'RandomForestClassifier', 'best_params': {'max_depth': 5, 'n_estimators': 100}, 'accuracy': 0.8227848101265823, 'avg_score': 0.89703989703989

C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py:547: FitFailedWarning: 
180 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
60 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Asus\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packa

In [32]:
print(f"best acc= {best_accuracy}\nbest test_size= {best_test_size}\nbest smote= {best_smote}")

best acc= 0
best test_size= 0.25000000000000006
best smote= 0.9


In [29]:
np.arange(0.1, 0.3, 0.05)

array([0.1 , 0.15, 0.2 , 0.25])